# A (very) short primer on Bayesian vs. Frequentist regression

# LPO Quantitative Methods Colloquium | 11 April 2017
### Benjamin Skinner & Will Doyle
#### [GitHub Repository](https://github.com/btskinner/lpoqmw/tree/master/apr_11_2017)

In [ ]:
## libraries and jupyter-specific options
libs <- c('tidyverse','lme4','plotly','rstan','rstanarm','shinystan')
suppressMessages(invisible(lapply(libs, require, character.only = TRUE)))
options(width = 120, warn = -1)

# Overview of Bayesian inference

[![](https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png)](https://xkcd.com/1132/)
[![](https://imgs.xkcd.com/comics/seashell.png)](https://xkcd.com/1236/)

 ### Paper 

> Gill, Jeff and Christopher Witko (2013) "[Bayesian Analytical Methods: A Methodological Prescription for Public Administration](https://academic.oup.com/jpart/article-abstract/23/2/457/1003493/Bayesian-Analytical-Methods-A-Methodological?redirectedFrom=fulltext)." *Journal of Public Administration Research and Theory*, 23 (2), 457-494. doi: [10.1093/jopart/mus091](https://doi.org/10.1093/jopart/mus091)

# Methods to compute posterior distributions

## Analytic (closed-form solutions)

### EXAMPLE: Repeated binary trials (*i.e.,* coin flips)

**Likelihood**: $p(X \mid \theta) \sim $ [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution)
$$ p(X \mid \theta) = {n \choose k}\theta^k(1-\theta)^{n-k}\tag{$k$ successes in $n$ trials} $$

**Prior**: $p(\theta)\sim $[Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution)
$$ p(\theta) = \frac{1}{Beta(\alpha,\beta)}\theta^{\alpha - 1}(1-\theta)^{\beta - 1} \tag{$\alpha$ successes, $\beta$ failures} $$

**Posterior**: $p(\theta \mid X)\sim $ [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution)
$$ p(\theta \mid X) \propto {n \choose k}\theta^k(1-\theta)^{n-k} \times \frac{1}{Beta(\alpha,\beta)}\theta^{\alpha - 1}(1-\theta)^{\beta - 1} $$

$$ p(\theta \mid X) \propto \theta^k(1-\theta)^{n-k} \times \theta^{\alpha - 1}(1-\theta)^{\beta - 1} $$

$$ p(\theta \mid X) \propto \theta^{k + \alpha - 1}(1-\theta)^{n - k + \beta - 1} $$

$$ p(\theta \mid X) \sim Beta(\alpha + k, \beta + n - k) $$

In [ ]:
## data (k = success; n = total trials)
x <- c(rep(1, 250), rep(0, 150))
k <- sum(x == 1)
n <- length(x)

## prior hyperparameters (a = success; b = failure)
a <- 100
b <- 100

# ## prior, likelihood, posterior: random draws
prior <- rbeta(n, a, b)
likelihood <- rbinom(n, n, k/n)
posterior <- rbeta(n, a + k, b + n - k)

## put into data frame
a_bayes <- data.frame(theta = c(prior, likelihood / n, posterior),
                     distribution = c(rep('prior', length(prior)),
                                     rep('likelihood', length(likelihood)),
                                     rep('posterior', length(posterior)))) %>%
            mutate(distribution = as.factor(distribution))

## plot
ggplot(a_bayes, aes(x = theta, fill = distribution)) + 
    geom_density(alpha = 0.25, adjust = 2) +
    ylab('') +
    xlab(expression(theta))

### Analytic computation: pros and cons

**Pros**
1. Sometimes very easy to compute
2. Gives the clearest description of the posterior

**Cons**
1. Algebra + [knowledge of conjugate priors](https://en.wikipedia.org/wiki/Conjugate_prior)
2. Almost never available for interesting problems

## Markov Chain Monte Carlo (MCMC) Sampling

For most interesting problems, there isn't an analytic or closed form solution to the posterior distribution. Or it's a pain to compute. In these situations one can try to sample directly from the posterior distribution by (1) choosing a computational algorithm and (2) relying on [Markov Chain Monte Carlo (MCMC) theory](http://www.mcmchandbook.net/HandbookChapter1.pdf) to make inferences. MCMC theory is an entire topic unto itself, so we won't delve into it today. Suffice to say that it is what permits us to feel confident that under certain conditions, our samples represent a good/useful approximation of the true posterior.

### MCMC alogorithms in terms of Bayesian analysis (50k feet view of the procedure)
1. Decide upon method for choosing a value of an unknown parameter ($\beta$ in a regression, for example)
2. Have rule for deciding whether value is a good choice and store (or not) accordingly

#### Procedure

To produce $S$ samples from the posterior:
    
    1. Initialize all unknowns (e.g., regression coefficients) with some starting values  
    while (s < S):
        2. Choose new parameter
        3. Implement decision rule:
            1. Keep new parameter if rule says it's good
            2. Drop new parameter if rule says it's bad and keep old one instead
        4. Store value based on decision

As always, the devil is in the details. Specifically:  
1. *What's a good way to generate new values for parameters?*  
2. *What's a good rule for deciding whether to keep the new parameter?*

## MCMC samplers and swimming pools

There are many, many methods for sampling from the posterior. The following three are the most common. We won't go into the math behind each, but those who are interested can check out the first chapter of the [MCMC Handbook](http://www.mcmchandbook.net/HandbookChapter1.pdf). Conceptualizing a multidimensional distribution is difficult, so for purposes of illustration, we'll limit ourselves to a two-dimensional distribution that has a single maximum point. 

In [ ]:
## create data that are distributed bivariate normal
mu <- c(0,0)
Sigma <- matrix(c(1,0,0,1),2,2)
beta <- MASS::mvrnorm(200, mu, Sigma)
dens <- MASS::kde2d(beta[,1], beta[,2], n = nrow(beta))

## interactive plot
axx <- list(title = 'beta_1')
axy <- list(title = 'beta_2')
axz <- list(title = 'Joint density')
p <- plot_ly(x = dens$x, y = dens$y, z = dens$z) %>% 
    add_surface() %>%
    layout(scene = list(xaxis = axx, yaxis = axy, zaxis = axz))
embed_notebook(p, height = '650px', file = 'figures/bivariate.html')

The joint distribution of the parameters, $ \beta_1 $ and $ \beta_2 $, is most likely at the peak (yellow). For purpose of visualizing our sampler, let's flip the distribution upside down. Now the joint likelihood is greatest at the bottom: a swimming pool!

The goal of the sample is describe the swimming pool by moving around the pool and taking snapshots along the way. We are interested in the deepest part of the pool because that's the point at which our unknown parameters are most likely. But since we're Bayesians now, we're also interested in the parts that aren't quite as likely. (Just because are they less likely doesn't mean they *won't* ever occur.) We want to describe the full distribution...the full pool.

### Gibbs Sampler

The [Gibbs Sampler](https://en.wikipedia.org/wiki/Gibbs_sampling) is a compromise between an analytic and a computational approach. Think of a posterior with multiple unknowns, for example, a normal linear regression where $\beta$ and the regression variance term, $\sigma^2$, are unknown. The full joint posterior, $ p(\beta, \sigma^2 \mid X) $, may be intractable or difficult to solve. On the other hand, the marginal posterior distributions of the terms, $ p(\beta \mid X, \sigma^2) $ and $ p(\sigma^2 \mid X, \beta) $, may be more tractable.

The Gibbs sampling algorithm therefore uses a bit of algebra to rearrange the full posterior into a set of marginal posteriors, one for each unknown or group of unknowns. The algorithm then follows this procedure:

```
1. initialize all parameters
for (s < S):
    2. update first unknown
    3. update second unknown
    ...
    4. update last unknown
    5. store all updates
```

Because the marginal posteriors are solveable, all new parameter choices are selected when updating.

#### Gibbs and the pool

Thinking about our swimming pool, it works like this:

1. Start at some arbitraty point in your pool
2. Pick one dimension of the pool (width- or length-wise)
3. Draw a straight line along that dimension, one end to the other
4. Move to the point on that line that is at the lowest point of the pool
5. Now choose the other dimension (perpendicular to the first) and draw a straight line along it
6. Move again (perpendicular to the first move) until you reach the lowest point along that line
7. Repeat until you stop moving that much or for an arbitrary number of steps

Eventually, no matter where you start, you should reach the deepest point. Under some conditions, the Gibbs sampler can find the posterior maximum (deepest part of the pool) pretty quickly. It may do so, however, at the expense of really exploring the full posterior/pool.

### Metropolis-Hastings

The [Metropolis-Hastings](https://en.wikipedia.org/wiki/Metropolis–Hastings_algorithm) sampler doesn't try to compute marginal posteriors. Instead, it selects new parameters from a proportional candidate distribution, compares a function of these parameters to a function of the old parameters, and accepts/rejects based on a probabilistic decision rule. The steps for MH in a regression framework are:

```
1. initialize all parameters
for (s < S):
    2. choose new parameters from candidate distribution
    3. compute the log likelihood (ll) of the data using these parameters
    4. compute ratio: ll_new / ll_old = alpha
    5. accept with probability: min(1, alpha)

```

The MH sampler will always accept new values that improve the fit (increase the log-likelihood). It won't necessarily reject those that reduce the log-likelihood, however. If they are close ($\alpha = .98$), they are quite likely to be accepted. On rare occasions, they will be accepted even when they are not that great. This feature allows the sampler to "jump around" and (hopefully) not become stuck in local maxima.

#### MH and the pool

Returning to the pool, Metropolis-Hastings works like this:

1. Pick some inital values for all your unknowns
2. Pick another pool that you think has a similar shape* to your pool that:  
    1. covers your pool (is bigger than)  
    2. you can describe well (you know what it looks like)
3. Move the point in the target pool that your random parameters give you
4. Compare the depth of this point to the depth at the old point  
    1. if deeper record this point  
    2. if shallower, get a random number between 1 and 100 and divide by 100
        1. if number is < ratio of new depth over old depth, record new value
        2. if number is > ratio of new depth over old depth, record old value again
5. Repeat for a determined number of steps

\* The shape of the proposal pool doesn't have to match the target pool exactly. If it can be transformed to fit the shape of the proposal pool, that will work, too. 

Unlike the Gibbs sampler, the Metropolis-Hastings sampler is more likely to explore more of the pool since it jumps around. What if the shallow end of the pool isn't smoothly sloping but rather has an indentation? The Gibbs sample could become stuck here, but the MH can jump out. The MH sampler requires, however, that the proposal pool fully "contain" the target. If it doesn't, there will be some places of the target pool that you'll never visit because they aren't part of the proposal pool. The MH sampler also requires some tuning so that it gives good proposal values. Too many bad ones (outside the target pool), will mean that you don't move much. Too many that are accepted and it will be very slow, taking random baby steps around the pool.

### Hamiltonian Monte Carlo

The Hamilton Monte Carlo sampler is the hot new sampler. It uses [Hamiltonian mechanics](https://en.wikipedia.org/wiki/Hamiltonian_mechanics) to simulate a particle moving around the posterior parameter space. The particle is given both potential and kinetic energy. Based on the gradient (shape) of the posterior, the particle is given direction. 

Imagine a hockey puck given a shove in a swimming pool. It will move around the pool, going up and down the sides. It may land in the shallow end quite often, but over time, it will more often end up in the deep end. Provided the algorithm can be tuned well (so the puck doesn't go flying outside of the pool), the HMC sampler can be very efficient at exploring the posterior parameter space.

Stan uses a variant of the HMC, the [No-U-Turn Sampler (NUTS)](http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf). Because the HMC particle can double back on itself, it's not quite as efficient as it could be (imagine shoving the hockey puck and after moving all over the pool, it landed two inches from your hand). NUTS uses some additional coding to make sure that the HMC doesn't double back: no U turns!

## Enough words...let's see them in action!

[Visualizations of various MCMC algorithms](http://chi-feng.github.io/mcmc-demo/)

# Coding examples: Bayesian vs. frequentist regression

## Data

Data come from the Kindergarten year of the Tennessee STAR class reduction experiment. The data can be downloaded from the [Harvard Dataverse archive](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/10766). The script to make the reduced data set used in these examples, `clean_data.R`, is in the [Github repository](https://github.com/btskinner/lpoqmw/tree/master/apr_11_2017).

In [ ]:
## read data
df <- read_csv('tn_star_k.csv', 
               col_types = cols(.default = col_integer(), 
                                math_std = col_double(),
                                read_std = col_double()))
head(df)

## Codebook

|Variable|Description|
|:--|:--|
|studnid | Unique student ID|
|female| == 1 if student is female|
|black| == 1 if student is Black|
|asian| == 1 if student is Asian|
|hispc| == 1 if student is Hispanic|
|natam| == 1 if student is Native American|
|orace| == 1 if student is another Race/ethnicity|
|t\_smc| == 1 if treatment is a small class|
|t\_rgc| == 1 if treatment is a regular class|
|t\_rga| == 1 if treatment is a regular class with a teacher's aide|
|incit| == 1 if student lives in inner city area|
|subur| == 1 if student lives in suburban area|
|urban| == 1 if student lives in urban area|
|rural| == 1 if student lives in rural area|
|frpr| == 1 if student receives free or reduced price lunch|
|spced| == 1 if student receives special education services|
|math| math test score|
|read| reading test score|
|math\_std| standardized math test score|
|read\_std| standardized reading test score|

# Single Level Models

** Formula **
$$ y_i = \beta_0 + \beta_{smc} \cdot T_i^{small\,class} + \beta_{aid} \cdot T_i^{aide} + X_i \beta_k + \varepsilon_i $$

## Frequentist

In [ ]:
## frequentist linear model
freq_lm <- lm(math_std ~ t_smc + t_rga + female + black + asian
              + hispc + natam + orace + incit + subur + urban
              + frpl + spced,
              data = df)

In [ ]:
## print
summary(freq_lm)

## Bayesian

In [ ]:
## show Stan model
writeLines(readLines('normal.stan'))

In [ ]:
## compile Stan model
norm_mod <- stan_model(file = 'normal.stan')

In [ ]:
## set up Stan data in list
y <- df$math_std
x <- df %>%
    select(t_smc, t_rga, female, black, asian,
           hispc, natam, orace, incit, subur, urban,
           frpl, spced) %>%
    as.matrix(.)
N <- nrow(x)
K <- ncol(x)

stan_df <- list('y' = y, 'x' = x, 'N' = N, 'K' = K)

In [ ]:
## Bayesian linear model using rstan
bayes_lm_1 <- sampling(norm_mod,
                       data = stan_df,
                       iter = 1000,
                       chains = 3,
                       cores = 3,
                       pars = c('alpha','beta','sigma','lp__'))

In [ ]:
## print
print(bayes_lm_1)

In [ ]:
## Bayesian linear model using rstanarm
bayes_lm_2 <- stan_glm(math_std ~ t_smc + t_rga + female + black + asian
                       + hispc + natam + orace + incit + subur + urban
                       + frpl + spced,
                       data = df,
                       family = gaussian(link = 'identity'),
                       chains = 3,
                       cores = 3,
                       iter = 1000)

In [ ]:
## print
summary(bayes_lm_2, digits = 2)

In [ ]:
## check convergence
# launch_shinystan(bayes_lm_1, quiet = TRUE)

# Varying intercept models

** Formula **
\begin{align} 
y_i &= \beta_j + \beta_{smc} \cdot T_i^{small\,class} + \beta_{aid} \cdot T_i^{aide} + X_i \beta_k + \varepsilon_i \\
\beta_j &\sim Normal(\mu, \sigma_j)
\end{align}

## Frequentist

In [ ]:
## varying intercept model
freq_vi <- lmer(math_std ~ -1 + t_smc + t_rga + female + black + asian
                + hispc + natam + orace + incit + subur + urban
                + frpl + spced + (1 | school),
                data = df)
summary(freq_vi)

## Bayesian

In [ ]:
## show Stan model
writeLines(readLines('vi_normal.stan'))

In [ ]:
## compile Stan model
vi_norm_mod <- stan_model(file = 'vi_normal.stan')

In [ ]:
## add information for school-level varying intercepts
school <- df$school
J <- length(unique(school))

stan_df <- list('y' = y, 'x' = x, 'N' = N, 'K' = K, 'J' = J, 'school' = school)

In [ ]:
## Bayesian varying intercept model using rstan
bayes_vi_1 <- sampling(vi_norm_mod,
                       data = stan_df,
                       iter = 2000,
                       chains = 3,
                       cores = 3,
                       pars = c('alpha_mu','alpha','beta',
                                'alpha_sigma','sigma','lp__'))

In [ ]:
## print results
print(bayes_vi_1)

In [ ]:
## Bayesian varying intercept model using rstanarm
bayes_vi_2 <- stan_glmer(math_std ~ -1 + t_smc + t_rga + female + black + asian
                         + hispc + natam + orace + incit + subur + urban
                         + frpl + spced + (1 | school),
                         data = df,
                         family = gaussian(link = 'identity'),
                         chains = 3,
                         cores = 3,
                         iter = 2000)

In [ ]:
## print results
summary(bayes_vi_2, digits = 2)

In [ ]:
## check convergence
# launch_shinystan(bayes_vi_1, quiet = TRUE)